<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/1_LimpezaDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install pyspark

In [ ]:
from google.colab import files
upload = files.upload()

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import pyspark
from pyspark.sql import SparkSession

In [57]:
spark = SparkSession.builder.getOrCreate()

In [62]:
produtos = spark.read.csv('produtos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('vendedores.csv', header=True, inferSchema=True)
clientes = spark.read.csv('clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv("itens_pedido.csv", header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('pagamentos_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('pedidos.csv', header=True, inferSchema=True)

In [61]:
!ls /content/

avaliacoes_pedido.csv  drive		 pagamentos_pedido.csv	produtos.csv  vendedores.csv
clientes.csv	       itens_pedido.csv  pedidos.csv		sample_data


In [55]:
ls /content/ grep

ls: cannot access 'grep': No such file or directory
/content/:
avaliacoes_pedido.csv  drive/            pagamentos_pedido.csv  produtos.csv  vendedores.csv
clientes.csv           itens_pedido.csv  pedidos.csv            sample_data/


In [63]:
print('dataframe produtos: \n', produtos.show(n=5, truncate=False))
print('dataframe vendedores: \n', vendedores.show(5))
print('dataframe clientes: \n', clientes.show(5))
print('dataframe itens_pedido: \n', itens_pedido.show(5))
print('dataframe pagamentos_pedido: \n', pagamentos_pedido.show(5))
print('dataframe avaliacoes_pedido: \n', avaliacoes_pedido.show(5))
print('dataframe pedidos: \n', pedidos.show(5))

+--------------------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|id_produto                      |categoria_produto    |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff4541ed26657ea517e5|perfumaria           |40                  |287                      |1                       |225           |16                    |10               |14                |
|3aa071139cb16b67ca9e5dea641aaa2f|artes                |44                  |276                      |1                       |1000          |30                    |18               |

In [64]:
# Acessar Colunas
from pyspark.sql.functions import col

clientes.select('id_cliente').show(n=1, truncate=False)
clientes.select(col('id_cliente')).show(1)
clientes.select(clientes['id_cliente']).show(1)
clientes.select(clientes.id_cliente).show(1)

+--------------------------------+
|id_cliente                      |
+--------------------------------+
|06b8999e2fba1a1fbc88172c00ba8bc7|
+--------------------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row



In [65]:
produtos_tratar_categ_nulos = produtos.na.fill({'categoria_produto': ' Não especificado'})

produtos_tratar_categ_nulos.filter(col('categoria_produto') == ' Não especificado').count()

610

In [ ]:
!

In [68]:
print('Total de pedidos: ', pedidos.count())

pedidos_unicos = pedidos.dropDuplicates()
print('Total de pedidos únicos: ', pedidos_unicos.count())

pedidos_remocao_nulos = pedidos_unicos.na.drop()
print('Total de peidos após limpeza de valores nulos: ', pedidos_remocao_nulos.count())

pedidos_remocao_nulo_id = pedidos_unicos.na.drop(subset=['id_cliente', 'id_pedido'])
print('Total de pedidos após limpeza de valores nulos na coluna id_pedido: ', pedidos_remocao_nulo_id.count())

Total de pedidos:  99441
Total de pedidos únicos:  99441
Total de peidos após limpeza de valores nulos:  96461
Total de pedidos após limpeza de valores nulos na coluna id_pedido:  99441


In [69]:
colunas =["peso_produto_g", "comprimento_produto_cm", "altura_produto_cm", "largura_produto_cm"]

for coluna in colunas:
  produtos = produtos.na.fill({coluna: 0})

In [71]:
produtos.write.mode('overwrite').option("header", "true").csv('/content/drive/MyDrive/Colab Notebooks/spark/produtos_tratada_csv')

In [72]:
produtos.write.mode('overwrite').option("header", "true").csv('produtos_tratada_csv')

In [74]:
spark.read.option('header', 'true').csv('/content/drive/MyDrive/Colab Notebooks/spark/produtos_tratada_csv').show(5)

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  

In [73]:
spark.read.option('header', 'true').csv('produtos_tratada_csv').show(5)

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  